<a href="https://colab.research.google.com/github/yuna1998/datamining/blob/main/ensembleipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
library(MASS)

In [2]:
install.packages("caret")
install.packages("caretEnsemble")
install.packages("doParallel")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘numDeriv’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘iterators’, ‘gower’, ‘ipred’, ‘timeDate’, ‘foreach’, ‘plyr’, ‘ModelMetrics’, ‘reshape2’, ‘recipes’, ‘pROC’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘pbapply’, ‘gridExtra’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
library(caret)
library(caretEnsemble)
library(doParallel)

Loading required package: lattice

Loading required package: ggplot2

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”

Attaching package: ‘caretEnsemble’


The following object is masked from ‘package:ggplot2’:

    autoplot


Loading required package: foreach

Loading required package: iterators

Loading required package: parallel



In [4]:
nc = detectCores()
registerDoParrel(nc)

ERROR: ignored

In [ ]:
n = dim(Boston)[1]

In [ ]:
set.seed(1)

In [ ]:
train = sample(n, n/2)
X_train = Boston[train, -14]
y_train = Boston[train, 14]
X_test = Boston[-train, -14]
y_test = Boston[-traon, 14]

In [ ]:
my_control = trainControl(method = "cv", number = 10, savePredictions = "final", allowParallel =TRUE )

In [ ]:
set.seed(2)

In [ ]:

model_list = caretList(X_train, y_train, methodList = c("lm", "rf", "xgbTree", "xgbLinear", "svmRadial"),
preProcess = c("center", "scale"), trConrol = my_control)

In [ ]:
rs = resample(model_list)
rs$values
dotplot(rs, metric = "RMSE")
modelCor(rs) #model correlations if high then not so much imporvement by ensemble

In [ ]:
#test RMSE
p_lm = predeict.train(model_list$lm, newdata = X_test)
p_rf = predeict.train(model_list$rf, newdata = X_test)
p_xgbTree = predeict.train(model_list$xgbTree, newdata = X_test)
p_xgbLinear = predeict.train(model_list$xgbLinear, newdata = X_test)
p_svmRadial = predeict.train(model_list$svmRadial, newdata = X_test)

In [ ]:
RMSE(p_lm, y_test)
RMSE(p_rf, y_test)
RMSE(p_xgbTree, y_test)
RMSE(p_xgbLinear, y_test)
RMSE(p_svmRadial, y_test)

In [ ]:
#stacked regression using a linear model
set.seed(3)
ens1 = caretEnsemble(model_list, metric = "RMSE", trConrol = my_control)
print(ens1)
summary(ens1)

In [ ]:
p_ens1 = predict(ens1, newdata = X_test)
results = data.frame(
    lm = RMSE(p_lm, y_test),
    rf = RMSE(p_rf, y_test),
    xgbTree = RMSE(p_xgbTree, y_test),
    xgbLinear = RMSE(p_xgbLinear, y_test),
    svmRadial = RMSE(p_svmRadial, y_test) ,
    stacked_regression = RMSE(p_ens1, y_test)
)


In [ ]:
#other meta learner  : elastic net : mixture of ridge and LASSO
set.seed(4)
ens2 = caret$Stack(model_list, method = "glmnet", trControl = my_control,
matric = "RMSE")
ens2
p_ens2 = predict(ens2, newdata = X_test)
RMSE(p_ens2, y_test)
results$stacking_elastic_net = RMSE(p_ens2, y_test)
results

In [ ]:
set.seed(4)
ens3 = caret$Stack(model_list, method = "gbm", trConrol = my_contol, metric = "RMSE")
ens3
p_ens3 = predict(ens3, newdata = X_test)
RMSE(p_ens3, y_test)

In [ ]:
set.seed(4)
ens3 = caret$Stack(model_list, method = "rf", trConrol = my_contol, metric = "RMSE")
ens3
p_ens3 = predict(ens3, newdata = X_test)
RMSE(p_ens3, y_test)

In [ ]:
#Level-1 data
z1 = model_list$lm$pred[order(model_list$lm$pred$rowIndex), ]$pred #cv  predicted value with original order
z2 = model_list$rf$pred[order(model_list$rf$pred$rowIndex), ]$pred
z3 = model_list$xgbTree$pred[order(model_list$xgbTree$pred$rowIndex), ]$pred
z4 = model_list$xgbLinear$pred[order(model_list$xgbLinear$pred$rowIndex), ]$pred
z5 = model_list$svmRadial$pred[order(model_list$svmLinear$pred$rowIndex), ]$pred

In [ ]:
Z_train = data.frame(z1 = z1, z2 = z2, z3 = z3, z4 = z4, z5 = z5) #level-1 data
head(Z_train)
fit_ens1 = lm(y_train~.,data=Z_train)
summary(fit_ens1)
coef(fit_ens1)
summary(ens1)

In [ ]:
#multilevel stacking
ZX_train = cbind(Z_train,X_train)
head(ZX_train)
fit_m = train(ZX_train, y_train, method = "glmnet", metric = "RMSE", trControl = my_control)
fit_m
p_lm = predeict.train(model_list$lm, newdata = X_test)
p_rf = predeict.train(model_list$rf, newdata = X_test)
p_xgbTree = predeict.train(model_list$xgbTree, newdata = X_test)
p_xgbLinear = predeict.train(model_list$xgbLinear, newdata = X_test)
p_svmRadial = predeict.train(model_list$svmRadial, newdata = X_test)

Z_test = data.frame(z1~p_lm, x2~p_rf, z3~p_xgbTree, z4~p_xgblinear, z5~p_svmRadial)
ZX_test = cbind(Z_test, X_Test)

p_ml = predict.train(fit_m, newdata = ZX_test)
RMSE(p_ml, y_test)
results$multi_level_stack = RMSE(p_ml, y_test)
results 

In [ ]:
#Stacking for classification
hdata = read.csv(file = "~~", header = T, stringAsFactors = T)
hdata$X = NULL
str(hdata)
#check missing
sum(is.na(hdata))
#small number of missing --> remove them

hdata = na.omit(hdata)
n = dim(hdata)[1]
n
#13 predictors and a binary response
set.seed(10)
train = sample(n, n/2)
htrain = hdata[train,]
htest = hdata[-train,]

In [ ]:
dim(hdata)
X.train = htrain[,-14]
X.test = htest[.-14]
y.train = htrain[,14]
y.test = htest[,14]

my.control = trainControl(method = "cv", number=10, savePredictons = "final", allowParallel = TRUE, classProbs = TRUE)
set.seed(2)
hmodel = careList(X.train, y.train, methodList = c("rf", "glm", "naive_bayes", metric ="Accuracy", trConrol = my.control))
hmodel


hmodel$rf
hmodel$glm
hmodel$naive_bays


pc_hrf = predict.train(hmodel$rf, X.test)
pc_hglm = predict.train(hmodel$glm, X.test)
pc_hnb = predict.train(hmodel$naive_bayes, X.test)

acc = function(x,y) mean(x==y)
acc(pc_hrf, y.test)
acc(pc_hglm, y.test)
acc(pc_hnb, y.test)


In [ ]:
##What is level-1 data
head(hmodel$rf$pred)
z1 = model$rf$pred[order(hmodel$rf$pred$rowIndex),]$Yes #CV posterior porb iwth the same index
z2 = model$glm$pred[order(hmodel$glm$pred$rowindex),]$Yes
z3 = model$naive_bayes$pred[order(hmodel$naive_bayes$pred$rowindex),]$Yes

Z.train = data.frame(z1~z1, z2~z2, z3~z3)

Z.test = data.frame(z1~predict.train(hmodel$rf, X.test, "prob")$Yes,
                    z2~predict.train(hmodel$glm, X.test, "prob")$Yes,
                    z3~predict.train(hmodel$naive_bayes, X.test, "prob")$Yes)

head(Z.test)
set.seed(3)
hens1 = caretEnsmble(hmodel, metrix = "Accuracy", trControl = my.control)
summary(hens1)

pc_hens1 = predict(hens1, X.test)
acc(pc_hens1, y.test)
res = data.frame(rf~acc(pc_hrf, y.test),
                glm~acc(pc_hglm, y.test),
                naive_bayes ~ acc(pc_hnb, y.test),
                stacking1 = acc(pc_hens1, y.test))
res
 

In [ ]:
set.seed(4)
hens2 = caret$Stack(hmodel,method = "glmnet" metric = "Accuracy", trConrol = my.control)
hens2

pc_hens2 = predict(hens2, X.test)
acc(pc_hens2, y.test)

res$stack_elastic_net = acc(pc_hens2, y.test)
res

In [ ]:
#multi-level stacking
set.seed(2222)
hens2 = caret$Stack(hmodel,method = "glmnet" metric = "Accuracy", trConrol = my.control, tuneLength = 5)
hens2

pc_hens2 = predict(hens2, X.test)
acc(pc_hens2, y.test)

res$stack_elastic_net = acc(pc_hens2, y.test)
res


ZX.train = cbing(Z.train, X.train)
ZX.test = cbind(Z.test, X.test)

head(ZX.train)
set.seed(10)
fit1 = train(model.matrix(~., -`, data = ZX.train), y.train, method = "glmnet", trControl = my.control)
pc_hm = predict.train(fit1, newdata = model.matrix(~.,-1, data = ZX.test))
acc(pc_hm, y.test)

cor(Z.train)
